# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,75.25,73,100,12.66,CK,1731629853
1,1,edinburgh of the seven seas,-37.0676,-12.3116,60.40,92,92,20.42,SH,1731629854
2,2,bardai,11.6500,7.2333,70.54,22,0,9.37,NG,1731629855
3,3,walvis bay,-22.9575,14.5053,60.53,84,3,4.21,NaN,1731629857
4,4,hawaiian paradise park,19.5933,-154.9731,78.31,80,100,12.66,US,1731629858


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_filter=city_data_df.loc[(city_data_df["Cloudiness"]==0) & (city_data_df["Max Temp"]>20) & (city_data_df["Max Temp"]<30) & (city_data_df["Humidity"]<70) & (city_data_df["Wind Speed"]<10)]
# Drop any rows with null values
# YOUR CODE HERE
city_data_filter= city_data_filter.dropna()
# Display sample data
# YOUR CODE HERE
city_data_filter

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
182,182,jiangzi,28.95,89.6333,21.81,57,0,4.03,CN,1731630069


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df= city_data_filter[["City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df[["Hotel Name"]]= ""
# Display sample data
# YOUR CODE HERE
hotel_df.head()

C:\Users\super\AppData\Local\Temp\ipykernel_3608\3149822245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df[["Hotel Name"]]= ""


,City,Country,Lat,Lng,Humidity,Hotel Name
182,jiangzi,CN,28.95,89.6333,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params ={
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat= row["Lat"]
    lng= row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jiangzi - nearest hotel: Yeti Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
182,jiangzi,CN,28.95,89.6333,57,Yeti Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)